# Token classification : Named entity recognition 

In [1]:
from datasets import load_dataset
raw_datasets = load_dataset("conll2003")

Reusing dataset conll2003 (/home/cgarcia37/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
raw_datasets


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3251
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3454
    })
})

In [3]:
raw_datasets["train"][0]["tokens"]


['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [4]:
raw_datasets["train"][0]["ner_tags"]      #labels as integers ready for training


[3, 0, 7, 0, 0, 0, 7, 0, 0]

In [5]:
ner_feature = raw_datasets["train"].features["ner_tags"]   # features attribute of our dataset
ner_feature

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None)

In [6]:
# O means the word doesn’t correspond to any entity.
# B-PER/I-PER means the word corresponds to the beginning of/is inside a person entity.
# B-ORG/I-ORG means the word corresponds to the beginning of/is inside an organization entity.
# B-LOC/I-LOC means the word corresponds to the beginning of/is inside a location entity.
# B-MISC/I-MISC means the word corresponds to the beginning of/is inside a miscellaneous entity.

label_names = ner_feature.feature.names        # label names by looking at the names attribute of the feature
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [7]:
words = raw_datasets["train"][0]["tokens"]         # idx=4 to see that “European Union” give a B-label for the first word and an I- label for the second
labels = raw_datasets["train"][0]["ner_tags"]
line1 = ""
line2 = ""
for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + " " * (max_length - len(word) + 1)
    line2 += full_label + " " * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


## Processing the data


In [8]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [9]:
# Tokenizers library must use a “fast” version. Check the current support in the library for each model
tokenizer.is_fast


True

In [10]:
# tokenize a pre-tokenized input --> string to numbers with BERT tokenizer

inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
inputs.tokens()
#inputs.__dict__

# mismatch between our inputs and the labels: the list of labels has only 9 elements, whereas our input now has 12 tokens.
# need to make sure we align all the labels with the proper words

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

In [11]:
inputs.word_ids()   #Return a list mapping the tokens to their actual word in the initial sentence for a fast tokenizer.

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

### Expand our label list to match the tokens

In [12]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]    # special tokens get a label of -100 --> ignored idx in the loss function
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [13]:
# Testing the function on our first sentence
labels = raw_datasets["train"][0]["ner_tags"]
word_ids = inputs.word_ids()
print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


## Dataset

In [14]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [15]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

Loading cached processed dataset at /home/cgarcia37/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee/cache-074ad689d2814c18.arrow
Loading cached processed dataset at /home/cgarcia37/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee/cache-118f8718e5b1b22f.arrow
Loading cached processed dataset at /home/cgarcia37/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee/cache-8d52db076f68351f.arrow


# Data collation

In [16]:
from transformers import DataCollatorForTokenClassification
# DataCollatorWithPadding pads the inputs (input IDs, attention mask, and token type IDs)

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [17]:
### testing --> 2 first samples

batch = data_collator([tokenized_datasets["train"][i] for i in range(3)])
batch["labels"]

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100],
        [-100,    5,    6,    6,    6,    0,    0,    0,    0,    0, -100, -100]])

In [18]:
 for i in range(3):
    print(tokenized_datasets["train"][i]["labels"]) 

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]
[-100, 5, 6, 6, 6, 0, 0, 0, 0, 0, -100]


## Metrics

In [19]:
#!pip install seqeval
from datasets import load_metric
metric = load_metric("seqeval")

In [20]:
#get the labels for our first training example

labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [21]:
# building a fake prediction --> just changing the value at index 2

predictions = labels.copy()
predictions[2] = "O"
predictions


['B-ORG', 'O', 'O', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [22]:
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

In [23]:
import numpy as np


def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

## Model

In [24]:
id2label = {str(i): label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [25]:
#id2label
#label2id

In [26]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [27]:
model.config.num_labels

9

## Fine-tuning

In [28]:
#from huggingface_hub import notebook_login
#notebook_login()


In [29]:
from transformers import TrainingArguments

args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False, # True need git-lfs
)

In [30]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

/home/cgarcia37/anaconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/home/cgarcia37/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:30: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
***** Running training *****
  Num examples = 14042
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.072391,0.886254,0.924436,0.904942,0.980441
2,0.188400,0.061302,0.917801,0.941434,0.929467,0.984341
3,0.048600,0.059711,0.918210,0.942780,0.930333,0.984797


***** Running Evaluation *****
  Num examples = 3251
  Batch size = 32
Saving model checkpoint to bert-finetuned-ner/checkpoint-439
Configuration saved in bert-finetuned-ner/checkpoint-439/config.json
Model weights saved in bert-finetuned-ner/checkpoint-439/pytorch_model.bin
tokenizer config file saved in bert-finetuned-ner/checkpoint-439/tokenizer_config.json
Special tokens file saved in bert-finetuned-ner/checkpoint-439/special_tokens_map.json
/home/cgarcia37/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/cgarcia37/anaconda3/lib/python3.8/site-packages/torch/nn/parallel/data_parallel.py:30: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 1 which
    has less than 75% of the memory or cores of GPU 0. You can do so by se

TrainOutput(global_step=1317, training_loss=0.09781042445131387, metrics={'train_runtime': 256.2309, 'train_samples_per_second': 164.406, 'train_steps_per_second': 5.14, 'total_flos': 1168207202281608.0, 'train_loss': 0.09781042445131387, 'epoch': 3.0})

# Inference

In [31]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "/home/cgarcia37/Teaching/bert-finetuned-ner/checkpoint-1317/"
token_classifier = pipeline(
    "token-classification", model=model_checkpoint, aggregation_strategy="simple"
)
token_classifier("Hello, I am Carlos Garcia and I work at the Data Science Institute of USF.")

loading configuration file /home/cgarcia37/Teaching/bert-finetuned-ner/checkpoint-1317/config.json
Model config BertConfig {
  "_name_or_path": "/home/cgarcia37/Teaching/bert-finetuned-ner/checkpoint-1317/",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "O",
    "1": "B-PER",
    "2": "I-PER",
    "3": "B-ORG",
    "4": "I-ORG",
    "5": "B-LOC",
    "6": "I-LOC",
    "7": "B-MISC",
    "8": "I-MISC"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "B-LOC": "5",
    "B-MISC": "7",
    "B-ORG": "3",
    "B-PER": "1",
    "I-LOC": "6",
    "I-MISC": "8",
    "I-ORG": "4",
    "I-PER": "2",
    "O": "0"
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers"

[{'entity_group': 'PER',
  'score': 0.9973402,
  'word': 'Carlos Garcia',
  'start': 12,
  'end': 25},
 {'entity_group': 'ORG',
  'score': 0.97571325,
  'word': 'Data Science Institute of',
  'start': 44,
  'end': 69},
 {'entity_group': 'ORG',
  'score': 0.9685546,
  'word': 'USF',
  'start': 70,
  'end': 73}]